In [6]:
import requests
import json
import pandas as pd

df = pd.read_csv('/Users/rajatsharma/Documents/Semester 2/Data Mining/Proj/simplify_all_jobs.csv')

csv_data = []

for index, row in df.iterrows():
    company_link = row['url_safe_slug']
    company_id = row['id']
    new_url = f"https://simplify.jobs/c/{company_link}"
    headers = {
        "Content-Type": "application/json",
        "Accept": "application/json, text/plain, /",
        "Origin": "https://simplify.jobs/",
        "Referer": "https://simplify.jobs/",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36"
    }

    for page in range(1, 2):
        url = "https://xv95tgzrem61cja4p.a1.typesense.net/multi_search?x-typesense-api-key=sUjQlkfBFnglUFcsFsZVcE7xhI8lJ1RG"
        payload = {
            "searches": [
                {
                    "query_by": "title,company_name,locations",
                    "per_page": 200,  
                    "highlight_full_fields": "title,company_name,locations",
                    "collection": "jobs",
                    "q": "",
                    "facet_by": "company_id,degrees,experience_level,functions,locations",
                    "filter_by": f"company_id:={company_id}",
                    "max_facet_values": 10,
                    "page": 1
                },
                {
                    "query_by": "title,company_name,locations",
                    "per_page": 200,  
                    "highlight_full_fields": "title,company_name,locations",
                    "collection": "jobs",
                    "q": "",
                    "facet_by": "company_id",
                    "max_facet_values": 10,
                    "page": 1
                }
            ]
        }

        response = requests.post(url, headers=headers, data=json.dumps(payload))

        if len(response.json()['results'][0]['hits']) > 0:
            companyNameInList = response.json()['results'][0]['hits'][0]["document"]["company_name"]
            numOfJobsInCompany = response.json()['results'][0]['found']
            location = response.json()['results'][0]['hits'][0]["document"]["geolocations"]
            date = response.json()['results'][0]['hits'][0]["document"]["start_date"]

            csv_data.append([companyNameInList, numOfJobsInCompany, location])

            for hit in response.json()['results'][0]['hits']:
                job_title = hit["document"]["title"]
                job_type = hit["document"]["type"]
                job_url = hit["document"]["url"]
                experience_level = ', '.join(hit["document"]["experience_level"])
                csv_data.append(["", "", "", job_title, job_type, job_url, experience_level,date])

        else:
            csv_data.append([company_link, 0, "-","-","-","-","-"])

columns = ['Company Name', 'Number of Jobs', 'Location', 'Job Title', 'Job Type', 'Job URL', 'Experience Level','Date']
df = pd.DataFrame(csv_data, columns=columns)

df.to_csv('job_data.csv', index=False)